## The Kanban Board Overview

#### Code for the User and user authentication service was obtained and modified from [This tutorial](https://github.com/CoreyMSchafer/code_snippets/tree/master/Python/Flask_Blog)

### **Implemented features:**
- One can add participants to their board
- One can add assignees to a task
- Drag-and-drop for state change
- Edit process for state change
- Universal UI principles for design implementation
- Username reference system for easy referencing in assigning and adding to board processes.
- Email-based password reset service
- Role-based board participant definition

### **User roles:**
#### ADMIN:
- Can add participant to board and edit board details
- Can add assignees to tasks and change task states through both drag-and-drop and the task edit process.
- Can edit, add, or delete tasks as they see necessary.
- Can remove participants from the board or change roles but not theirs

#### EDITOR:
- Can add assignees to tasks and change task states through both drag-and-drop and the task edit process.
- Can edit, add, or delete tasks as they see necessary.

#### VIEW-ONLY:
- Can only view the board but can still be assigned tasks but cannot change task state

## Test Users

Username: Adakole

Email: test@minerva.edu

Password: Test123

Boards: 2

---------------------
Username: cs162tests

Email: test@uni.minerva.edu

Password: Test123

Boards: 1

---------------------
Username: Samuel

Email: cs162tests@minerva.kgi.edu

Password: Test123

Boards: 1



### Register Users

In [ ]:
@main.route("/register", methods=['GET', 'POST'])
def register():
    if current_user.is_authenticated:
        return redirect(url_for('main.home'))
    form = RegistrationForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data).decode('utf-8')
        user = User(username=form.username.data, email=form.email.data, password=hashed_password)
        db.session.add(user)
        db.session.commit()
        flash('Your account has been created! You are now able to log in', 'success')
        return redirect(url_for('users.login'))
    elif request.method == "GET":
        return render_template('register.html', title='Register', form=form)
    else:
        return render_template('register.html', title='Register', form=form), 401

### Login Users

In [ ]:
@app.route("/login", methods=['GET', 'POST'])
def login():
    if current_user.is_authenticated:
        return redirect(url_for('home'))
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(email=form.email.data).first()
        if user and bcrypt.check_password_hash(user.password, form.password.data):
            login_user(user, remember=form.remember.data)
            next_page = request.args.get('next')
            return redirect(next_page) if next_page else redirect(url_for('home'))
        else:
            flash('Login Unsuccessful. Please check email and password', 'danger')
    elif request.method == "GET":
        return render_template('login.html', title='Login', form=form)
    else:
        return render_template('login.html', title='Login', form=form), 401

### Logout Users

In [ ]:
@app.route("/logout")
def logout():
    logout_user()
    return redirect(url_for('users.login')), 200

### Modify User account information

In [ ]:

def save_picture(form_picture):
    random_hex = secrets.token_hex(8)
    _, f_ext = os.path.splitext(form_picture.filename)
    picture_fn = random_hex + f_ext
    picture_path = os.path.join(app.root_path, 'static/profile_pics', picture_fn)

    output_size = (125, 125)
    i = Image.open(form_picture)
    i.thumbnail(output_size)
    i.save(picture_path)

    return picture_fn

In [ ]:
@app.route("/account", methods=['GET', 'POST'])
@login_required
def account():
    form = UpdateAccountForm()
    if form.validate_on_submit():
        if form.picture.data:
            picture_file = save_picture(form.picture.data)
            current_user.image_file = picture_file
        current_user.username = form.username.data
        current_user.email = form.email.data
        db.session.commit()
        flash('Your account has been updated!', 'success')
        return redirect(url_for('account'))
    elif request.method == 'GET':
        form.username.data = current_user.username
        form.email.data = current_user.email
    image_file = url_for('static', filename='profile_pics/' + current_user.image_file)
    return render_template('account.html', title='Account',
                           image_file=image_file, form=form)

### Manage reset email page

In [ ]:
def send_reset_email(user):
    token = user.get_reset_token()
    msg = Message('Password Reset Request',
                  sender='noreply@kanban.com',
                  recipients=[user.email])
    msg.body = f'''To reset your password, visit the following link:
{url_for('reset_token', token=token, _external=True)}
If you did not make this request then simply ignore this email and no changes will be made.
'''
    mail.send(msg)

In [ ]:
@app.route("/reset_password", methods=['GET', 'POST'])
def reset_request():
    if current_user.is_authenticated:
        return redirect(url_for('home'))
    form = RequestResetForm()
    if form.validate_on_submit():
        user = User.query.filter_by(email=form.email.data).first()
        send_reset_email(user)
        flash('An email has been sent with instructions to reset your password.', 'info')
        return redirect(url_for('login'))
    return render_template('reset_request.html', title='Reset Password', form=form)

### Manage reset token page

In [ ]:
@app.route("/reset_password/<token>", methods=['GET', 'POST'])
def reset_token(token):
    if current_user.is_authenticated:
        return redirect(url_for('home'))
    user = User.verify_reset_token(token)
    if user is None:
        flash('That is an invalid or expired token', 'warning')
        return redirect(url_for('reset_request'))
    form = ResetPasswordForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data).decode('utf-8')
        user.password = hashed_password
        db.session.commit()
        flash('Your password has been updated! You are now able to log in', 'success')
        return redirect(url_for('login'))
    return render_template('reset_token.html', title='Reset Password', form=form)

### Update a task because of addDrop

In [ ]:
@app.route('/update_task', methods = ["POST"])
def update_task():
    task_id = request.form["id"]
    new_state = request.form["state"]
    refr_task = Task.query.get(task_id)
    if refr_task:
        refr_task.state = new_state
        db.session.commit()
        flash("Task state has successfully changed!!", "success")
        return "Update has been made", 200
    else:
        flash("No valid task ID was supplied!!", "danger")
        return "No valid task ID was supplied", 401

### Remove a task from container directly

In [ ]:
@app.route('/remove_task', methods = ["POST"])
def remove_task():
    task_id = request.form["id"]
    refr_task = Task.query.get(task_id)
    if refr_task:
        db.session.delete(refr_task)
        db.session.commit()
        flash("Task has successfully been deleted!!", "success")
        return "That task has been deleted", 200
    else:
        flash("No valid task ID was supplied!!", "danger")
        return "No valid task ID was supplied", 401

### Create a new board

In [ ]:
@app.route("/new_board", methods = ["GET", "POST"])
@login_required
def new_board():
    form = BoardForm()
    if form.validate_on_submit():
        board_name = form.name.data
        board_description = form.description.data
        board_participants = form.participants.entries
        for participant in board_participants:
            participant.id = User.query.filter_by(username = participant.username).first()
        new_board = Board(name = board_name, description = board_description)
        db.session.add(new_board)
        db.session.flush()
        participants_list = [BoardParticipant(board_id = new_board.id, participant_id = current_user.id, role = BoardRolesEnum.ADMIN)] + [BoardParticipant(board_id = new_board.id, participant_id = participant.id, role = participant.role) for participant in board_participants if participant.id != current_user.id] 
        db.session.add_all(participants_list)
        db.session.commit()
        return redirect(url_for('board', board_id = new_board.id))
    elif request.method == "GET":
        return render_template("new_board.html", form = form, templateParticipant = ParticipantForm(prefix = "participants-_-"))
    else:
        return render_template("new_board.html", form = form, templateParticipant = ParticipantForm(prefix = "participants-_-")), 400

### Create a new task or update a task if id is provided

In [ ]:
@app.route("/task/<int:board_id>", methods = ["POST"])
@login_required
def task(board_id):
    form = TaskForm()
    pres_board = Board.query.get_or_404(board_id)
    form.assignees.choices = [(user.participant_id, user.participant.username) for user in pres_board.participants]
    tid = form.tid.data
    print(request.form)
    if form.validate_on_submit():
        task_name = form.name.data
        task_content = form.content.data
        task_state = form.state.data
        task_assignees = form.assignees.data
        assignees_list = [User.query.get(assignee) for assignee in task_assignees]
        assignees_list = [assignee for assignee in assignees_list if assignee is not None]
        roleCheck = BoardParticipant.query.get_or_404((board_id, current_user.id))
        if tid is None:
            if roleCheck.role > 1:
                new_task = Task(board_id = board_id, name = task_name, content = task_content, state = task_state, assignees = assignees_list)
                db.session.add(new_task)
                db.session.commit()  
                flash("The task has been created!!", "success")
            else:
                abort (403)
        else:
            task = Task.query.get(tid)
            if task:
                task.name = task_name
                task.content = task_content
                task.state = task_state
                task.assignees = assignees_list
                db.session.commit()
                flash("The task has been updated!!", "success")
    return redirect(url_for('board', board_id = board_id))

### Update a board or return a board dependent on the request method

In [ ]:
@app.route("/board/<int:board_id>", methods = ['GET', 'POST'])
@login_required
def board(board_id):
    boardForm = BoardForm()
    templateParticipant = ParticipantForm(prefix = "participants-_-")
    pres_board = Board.query.get_or_404(board_id)
    role_check = BoardParticipant.query.get((board_id, current_user.id))
    if role_check is None:
       abort (403) 
    if boardForm.validate_on_submit():
        if role_check.role < 3:
            abort(403)
        pres_board.name = boardForm.name.data
        pres_board.description = boardForm.description.data
        board_participants = boardForm.participants.entries
        pres_entries = pres_board.participants
        for entry in pres_entries:
            db.session.delete(entry)
        db.session.commit()
        new_participants = [BoardParticipant(board_id = pres_board.id, participant_id = current_user.id, role = BoardRolesEnum.ADMIN)] + list(set([BoardParticipant(board_id = pres_board.id, participant_id = participant.username.data, role = participant.role.data) for participant in board_participants if int(participant.username.data) != current_user.id]))
        for participant in new_participants:
            db.session.add(participant)
        db.session.commit()
        flash("Board has been updated!", "success")
        return redirect(url_for("board", board_id = board_id))
    elif request.method == "GET":    
        boardForm = BoardForm(name = pres_board.name, description = pres_board.description, participants = [ParticipantForm(username = participant.participant_id, role = participant.role) for participant in pres_board.participants])
    taskForm = TaskForm()
    tasks = [task.to_dict() for task in pres_board.tasks]
    name = pres_board.name
    for task in tasks:
        task["assignees"] = [assignee["id"] for assignee in task["assignees"]]
    todo = [task for task in tasks if task['state'] == "Requested"]
    doing = [task for task in tasks if task['state'] == "In Progress"]
    done = [task for task in tasks if task['state'] == "Completed"]
    taskForm.assignees.choices = [(user.participant_id, user.participant.username) for user in pres_board.participants]
    return render_template('board_page.html', todo = todo, doing = doing, done = done, taskForm = taskForm, boardForm = boardForm, role = role_check.role, board_id = board_id, templateParticipant = templateParticipant)